In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Input,Flatten
from keras.models import Model,load_model
import cv2

In [75]:
df = pd.read_csv(r"D:\Datasets\facial-keypoints-detection\training\training.csv")
df_test = pd.read_csv(r"D:\Datasets\facial-keypoints-detection\test\test.csv")


In [64]:
df['Image'] = df['Image'].apply(lambda x: np.fromstring(x, sep=' '))   # convert string into numpy array

In [65]:
df = df.dropna(axis=0)

In [66]:
df['Image'][0].shape

(9216,)

In [67]:
X = np.vstack(df['Image'].values) / 255.0  
X = X.astype(np.float32)
# return each images as 96 x 96 x 1
X = X.reshape(-1, 96, 96, 1)

y = df[df.columns[:-1]].values #(30 columns)
# Normalize the target value, scale values between 0 and 1
y = (y)/255.0
# shuffle train data 
X, y = shuffle(X, y, random_state=42)  
y = y.astype(np.float32)

In [68]:
X_train = X
y_train = y

In [69]:
# Creating Convolutional Neural Network

input_layer = Input(shape=(96,96,1))

conv_1 = Conv2D(filters=32,kernel_size=(5,5),activation='relu',name="conv_1")(input_layer)
max_pool_1 = MaxPool2D(pool_size=(2,2),name='max_pool_1')(conv_1)

conv_2 = Conv2D(filters=64,kernel_size=(3,3),activation='relu',name='conv_2')(max_pool_1)
max_pool_2 = MaxPool2D(pool_size=(2,2),name="max_pool_2")(conv_2)
dropout_2 = Dropout(0.1)(max_pool_2)

conv_3 = Conv2D(filters=128,kernel_size=(3,3),activation='relu',name="conv_3")(dropout_2)
max_pool_3 = MaxPool2D(pool_size=(2,2),name="max_pool_3")(conv_3)
dropout_3 = Dropout(0.2)(max_pool_3)

conv_4 = Conv2D(filters=256,kernel_size=(3,3),activation='relu',name="conv_4")(dropout_3)
max_pool_4 = MaxPool2D(pool_size=(2,2),name="max_pool_4")(conv_4)
dropout_4 = Dropout(0.3)(max_pool_4)

conv_5 = Conv2D(filters=256,kernel_size=(3,3),activation='relu',name="conv_5")(dropout_4)
max_pool_5 = MaxPool2D(pool_size=(2,2),name="max_pool_5")(conv_5)
dropout_5 = Dropout(0.1)(max_pool_5)


flatten = Flatten(name="flatten_layer")(dropout_5)

dense_1 = Dense(156,activation='relu', name="dense_1")(flatten)
dense_2 = Dense(128,activation='relu', name="dense_2")(dense_1)
dense_3 = Dense(64,activation='relu', name="dense_3")(dense_2)

dense_4 = Dense(30,name='dense_4')(dense_3)

In [70]:
model = Model(input_layer,dense_4)

In [71]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [72]:
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 96, 96, 1)]       0         
                                                                 
 conv_1 (Conv2D)             (None, 92, 92, 32)        832       
                                                                 
 max_pool_1 (MaxPooling2D)   (None, 46, 46, 32)        0         
                                                                 
 conv_2 (Conv2D)             (None, 44, 44, 64)        18496     
                                                                 
 max_pool_2 (MaxPooling2D)   (None, 22, 22, 64)        0         
                                                                 
 dropout_26 (Dropout)        (None, 22, 22, 64)        0         
                                                                 
 conv_3 (Conv2D)             (None, 20, 20, 128)       7385

In [73]:
model.fit(X_train,y_train,batch_size=200,epochs=100,verbose=1,validation_split=0.2)

Epoch 1/100
9/9 [==============================] - 3s 262ms/step - loss: 0.0259 - accuracy: 0.0123 - val_loss: 0.0123 - val_accuracy: 0.0000e+00
Epoch 2/100
9/9 [==============================] - 2s 245ms/step - loss: 0.0072 - accuracy: 0.5940 - val_loss: 0.0062 - val_accuracy: 0.6963
Epoch 3/100
9/9 [==============================] - 2s 249ms/step - loss: 0.0020 - accuracy: 0.3055 - val_loss: 0.0029 - val_accuracy: 0.2523
Epoch 4/100
9/9 [==============================] - 2s 247ms/step - loss: 8.0367e-04 - accuracy: 0.2442 - val_loss: 0.0025 - val_accuracy: 0.2523
Epoch 5/100
9/9 [==============================] - 2s 243ms/step - loss: 5.3383e-04 - accuracy: 0.3575 - val_loss: 0.0012 - val_accuracy: 0.2523
Epoch 6/100
9/9 [==============================] - 2s 246ms/step - loss: 3.4051e-04 - accuracy: 0.5824 - val_loss: 0.0016 - val_accuracy: 0.6963
Epoch 7/100
9/9 [==============================] - 2s 244ms/step - loss: 2.8937e-04 - accuracy: 0.6717 - val_loss: 0.0012 - val_accuracy: 

In [74]:
history = model.save("facial_detection_model_1.h5")

c:\Users\Omen\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
model = load_model(r"D:\vs code\python\DeepLearning\Projects\Facial_landmark_detection\facial_detection_model_1.h5")

In [5]:
image = cv2.imread(r"C:\Users\Omen\Desktop\image.jpg")

In [8]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

scaled_face = cv2.resize(gray, (96,96), 0, 0, interpolation=cv2.INTER_AREA)

In [10]:
print("Shape:", scaled_face.shape)
print("Data type:", scaled_face.dtype)

Shape: (96, 96)
Data type: uint8


In [9]:
points = model.predict(scaled_face)

ValueError: in user code:

    File "c:\Users\Omen\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Omen\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Omen\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Omen\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "c:\Users\Omen\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   